In [4]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time
import numpy as np
import cv2


In [ ]:

def get_images(self, lowS, lowV):
    self.image_getter.update_image()
    img = self.image_getter.cam_img
    img = cv2.resize(img, (widthImg, heightImg))                                                        # RESIZE IMAGE TO MAKE IT A SQUARE IMAGE
    imgThreshold = preProcessHSV(img, lowS, lowV)
    contours, _ = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)            # FIND ALL CONTOURS
    biggest, _ = biggestContour(contours)                                                               # FIND THE BIGGEST CONTOUR
    if biggest.size != 0:
        biggest = reorder(biggest)
        pts1 = np.float32(biggest)                                                                      # PREPARE POINTS FOR WARP
        pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]])                 # PREPARE POINTS FOR WARP
        matrix = cv2.getPerspectiveTransform(pts1, pts2)                                                # GER
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
        boxes = splitBoxes(imgWarpColored)
    return boxes

In [5]:
reachy = ReachySDK('localhost')

In [ ]:
goto(
    goal_positions=
            {reachy.r_arm.r_shoulder_pitch: 55,
            reachy.r_arm.r_shoulder_roll: -15,
            reachy.r_arm.r_arm_yaw: 0,
            reachy.r_arm.r_elbow_pitch: -85,
            reachy.r_arm.r_forearm_yaw: -10,
            reachy.r_arm.r_wrist_pitch: -50,
            reachy.r_arm.r_wrist_roll: 0},
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )

In [6]:
reachy.turn_off_smoothly('reachy')

In [7]:
import os
path = '/home/reachy/repos/TicTacToe/tictactoe/images/empty'
files = os.listdir(path)


for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index+813), '.jpg'])))

In [ ]:
reachy.turn_on('reachy')

In [ ]:
for f in reachy.fans.values():
    f.on()

In [ ]:
for f in reachy.fans.values():
    f.off()

In [ ]:
#GET dataset images
j = 1000
    #Save images for dataset
for i in range(9):
    boxes = board.get_images()
    cv2.imwrite('images/new/'+str(j)+'.jpg', boxes[i])
    j=j+1 

In [ ]:
i = 1
while (True):
    
    state = board.get_board()
    winner = board.get_winner(state)
    if winner == 'nobody':
        game = np.array_split(state,3)
        print(game[0])
        print(game[1])
        print(game[2])
        print('     ')
        print('     ')
        
        action =  -board.choose_next_action(state)[0] +9
        print(action)
        #print(action)
        robot.play_pawn(i,action)
        time.sleep(2)
        i = i + 1 
    elif winner==1:
        print('Robot won')
        break
    else:
        print('Human won')
        break
#robot.reachy.turn_off_smoothly('r_arm')


In [ ]:
    if winner == 'nobody':

        # if (board.incoherent_board_detected(state)):
        #     double_check_board = board.get_board(lowS, lowV)
        #     if np.any(double_check_board != state):
        #         print("RESTART") 
        #         #tictactoe_playground.shuffle_board()
        #         break

        game_inv = state[::-1]
        game_inv = np.array_split(game_inv,3)
        print(game_inv[0])
        print(game_inv[1])
        print(game_inv[2])
        print('       ')
        
        action =  - board.choose_next_action(state)[0] + 9
        #robot.play_pawn(i,action)
        time.sleep(2)
        
        i = i + 1 
    elif winner==1:
        print('Robot won')
        robot.reachy.turn_on('reachy')
        robot.happy()
        break
    else:
        print('Human won')
        robot.reachy.turn_on('reachy')
        robot.sad()
        break

robot.rest()
robot.reachy.turn_off('reachy')

In [ ]:
#game loop
while True:
    board = playground.get_board(lowS, lowV)
    
    if not reachy_turn:
        if playground.has_human_played(board, last_board):
            reachy_turn = True
        else:
            robot.run_random_idle_behavior()

    if (playground.incoherent_board_detected(board) or
                playground.cheating_detected(board, last_board, reachy_turn)):
        double_check_board = playground.get_board(lowS, lowV)
        if np.any(double_check_board != last_board): 
            #tictactoe_playground.shuffle_board()
            print("shuffle board")
            break
        else :
                # False detection, we will check again next loop
                continue
        
    if (not playground.is_final(board)) and reachy_turn:
        action, _ = playground.choose_next_action(board)
        board = robot.play(action, board)

        last_board = board
        reachy_turn = False
    
    if playground.is_final(board):
        winner = playground.get_winner(board)

        if winner == 1:
            robot.happy()
        elif winner == 2:
            robot.sad()
        else:
            robot.sad()
        print(winner)